In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import os
import pandas as pd
import datetime

from dotenv import load_dotenv

from data_chunking import datachunk
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint as HuggingFaceHub
from langchain_community.vectorstores.pinecone import Pinecone
from pinecone import Pinecone as pc
from pinecone import ServerlessSpec

In [3]:
load_dotenv()

True

In [20]:
def datachunk(transcripts_folder, pdf_folder):
    csv_files = [file for file in os.listdir(transcripts_folder) if file.endswith('.csv')]
    pdf_files = [file for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

    documents_csv = []
    documents_pdf = []

    # Extract text documents and add metadata from CSV files
    for csv_file in csv_files:
        try:
            transcripts_df = pd.read_csv(os.path.join(transcripts_folder, csv_file), delimiter=';')
        except FileNotFoundError:
            print(f"Error: The file {csv_file} was not found.")
            continue

        for index, row in transcripts_df.iterrows():
            document = Document(
                page_content=row["transcript_text"],
                metadata={
                    "speaker_name": row["speaker_name"],
                    "date_time": row["date_time"],
                    "source": csv_file
                }
            )
            documents_csv.append(document)

    print("CSV files loaded...")

    # Load text documents from .pdf files (keeping as they are)
    for pdf_file in pdf_files:
        loader = PyPDFLoader(os.path.join(pdf_folder, pdf_file))
        doc_content = loader.load()
        documents_pdf.extend(doc_content)

    print("PDF files loaded...")

    
    # Split text documents into chunks
    text_documents = [doc for doc in documents_pdf if isinstance(doc, Document) and doc.page_content]
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, length_function=len, )
    split_docs = text_splitter.split_documents(text_documents)

    # Combine split text documents with PDF documents
    docs = split_docs + [doc for doc in documents_pdf if not isinstance(doc, Document)]
    
    # Add data from CSV files to the combined documents
    docs.extend(documents_csv)
    
    return docs

In [21]:
transcripts_folder = os.path.join("..","data", "reformatted_transcripts")
pdf_folder = os.path.join("..","data", "EER-site-pages-pdf")

In [22]:
embeddings = HuggingFaceEmbeddings()
index_name = "eer-transcripts-pdfs"
pinecone_instance = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=embeddings)
spec = ServerlessSpec(cloud="aws",region="us-east-1")

c:\Users\szh\Desktop\EER\Bot-de-Continuonus\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
docs = datachunk(transcripts_folder, pdf_folder)
pinecone_instance.create_index(name=index_name, metric="cosine", dimension=768, spec=spec)

CSV files loaded...
PDF files loaded...


In [24]:
docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)